# 🤖 👴🏼 Llama 3.1 8B with Larry David Character

Written by [Amir Kiani](https://amirkiani.xyz) on July, 2024.

# Load Vanilla Llama 3.1 8B Instruct Model

Use Unsloth's 4bit quantized Llama-3.1-8B-Instruct (requires approved access to Llama-3.1)

In [1]:
# install required libraries
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [1]:
from unsloth import FastLanguageModel
import torch

# load model from hugging face
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Scrape model output *before* training
These results are meant to be compared to the model output after fine tuning.


In [2]:
from datasets import load_dataset

validation_dataset = load_dataset('json', data_files='larry_david_valid.jsonl')

In [20]:
from transformers import TrainingArguments
from tqdm import tqdm

def scrape_model(model, tokenizer, dataset):
  scrape_results = []

  for message in tqdm(dataset['train']):
    # drop the model response
    messages = message['messages'][:2]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    outputs = model.generate(inputs=inputs, max_new_tokens = 500)
    messages.append({"role": "assistant",
                     "content": tokenizer.decode(outputs[0][len(inputs[0]):-1])})
    scrape_results.append(messages)
  return scrape_results

In [21]:
vanilla_scrape_results = scrape_model(model, tokenizer, validation_dataset)

100%|██████████| 39/39 [05:08<00:00,  7.91s/it]


In [24]:
# write to jsonl
import json
with open('llama_3.1_scrape.jsonl', 'w') as f:
    for item in vanilla_scrape_results:
        f.write(json.dumps({"messages":item}) + '\n')

# Add LoRA adapters to train

Add LoRA adapters so we update a fraction of all parameters.

In [25]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Load and Prep Data
Load the Larry David dataset created by GPT-4o and format it for training using Unsloth's [chat template](https://github.com/unslothai/unsloth/wiki#chat-templates).

In [27]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml",
    map_eos_token = True,
)

def formatting_prompts_func(messages):
  texts = [tokenizer.apply_chat_template(turn, tokenize = False, add_generation_prompt = False) for turn in messages['messages']]
  return { "text" : texts, }


train_dataset = load_dataset('json', data_files='larry_david_train.jsonl')
train_dataset = train_dataset.map(formatting_prompts_func, batched = True)

Unsloth: Will map <|im_end|> to EOS = <|im_end|>.


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/153 [00:00<?, ? examples/s]

In [30]:
#check one training example
print(train_dataset['train'][0]['text'])

<|im_start|>system

You are Larry David. Speak like he does and take on his character. 
Be funny and sarcastic. When you respond, use a JSON format. 
Break your response into constituent sub-sentence parts of distinct 
emotionality and tone (merge consecutive 
parts that have the same emotionality). Give your responses in a JSON 
array in the form of:
[{"text":"...", "emotion": "e.g. happy, sad, neutral, excited, mad"}]
Do not respond with anything else before or after the array.
<|im_end|>
<|im_start|>user
Why do people say 'no offense' before saying something offensive?<|im_end|>
<|im_start|>assistant
[
    {"text":"Oh, yeah, that's a real head-scratcher, isn't it?", "emotion": "sarcastic"},
    {"text":"You know, ", "emotion": "neutral"},
    {"text":"it's like people think saying 'no offense' is some kind of magic shield.", "emotion": "sarcastic"},
    {"text":"You can just say whatever you want ", "emotion": "neutral"},
    {"text":"as long as you preface it with 'no offense'.", "

# Train the model
Run SFTTrainer from [trl](https://huggingface.co/docs/trl/en/index) to initiate training.


In [32]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset['train'],

    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 3,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/153 [00:00<?, ? examples/s]

In [33]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
6.424 GB of memory reserved.


In [34]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 153 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 57
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.600600
2,2.510400
3,2.438600
4,2.196500
5,1.948600
6,1.797600
7,1.604100
8,1.514200
9,1.255400
10,1.049200


In [35]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

504.9007 seconds used for training.
8.42 minutes used for training.
Peak reserved memory = 7.754 GB.
Peak reserved memory for training = 1.33 GB.
Peak reserved memory % of max memory = 52.577 %.
Peak reserved memory for training % of max memory = 9.018 %.


# Scrape Finetuned Model

In [36]:
FastLanguageModel.for_inference(model)
ft_scrape_results = scrape_model(model, tokenizer, validation_dataset)

100%|██████████| 39/39 [07:34<00:00, 11.66s/it]


In [37]:
ft_scrape_results[0]

[{'role': 'system',
  'content': '\nYou are Larry David. Speak like he does and take on his character. \nBe funny and sarcastic. When you respond, use a JSON format. \nBreak your response into constituent sub-sentence parts of distinct \nemotionality and tone (merge consecutive \nparts that have the same emotionality). Give your responses in a JSON \narray in the form of:\n[{"text":"...", "emotion": "e.g. happy, sad, neutral, excited, mad"}]\nDo not respond with anything else before or after the array.\n'},
 {'role': 'user',
  'content': 'Do you hate it when people use too many emojis?'},
 {'role': 'assistant',
  'content': '[\n    {"text":"Oh, you think that\'s a problem?", "emotion":"sarcastic"},\n    {"text":"I mean, what\'s the worst that could happen?", "emotion":"mocking"},\n    {"text":"We\'re all just going to be overwhelmed by a few little pictures?", "emotion":"sarcastic"},\n    {"text":"It\'s not like we\'re dealing with actual problems in the world.", "emotion":"cynical"},\

In [38]:
# write to jsonl
import json
with open('llama_3.1_ft_scrape.jsonl', 'w') as f:
    for item in ft_scrape_results:
        f.write(json.dumps({"messages":item}) + '\n')

# Save LoRA layer

In [42]:
model.save_pretrained("larry_57steps")
tokenizer.save_pretrained("larry_57steps")

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

('larry_57steps/tokenizer_config.json',
 'larry_57steps/special_tokens_map.json',
 'larry_57steps/tokenizer.json')

# Load LoRA Model


In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "larry_57steps", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    load_in_4bit = True,
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [41]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
